### Create maps in dense freesurfer space

In [1]:
from pathlib import Path
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
import numpy as np
from nibabel.freesurfer.io import read_geometry, read_label
from fmri_tools.surface.filter import LaplacianGaussian
from fmri_tools.surface.mesh import Mesh
from fmri_tools.io.surf import (
    read_mgh, write_mgh, write_label, patch_as_mesh, mgh_to_patch, curv_to_patch, 
    label_to_patch
    )
from src.config import DIR_BASE, SESSION
from src.helper import get_composed_label

SUBJ = "p1"
HEMI = "lh"
LAYER = 5
DIR_OUT = "/data/pt_01880/zzz_new"

# make output folders
DIR_DENSE = Path(DIR_OUT) / "dense"
DIR_FLAT = Path(DIR_OUT) / "flat"
DIR_DENSE.mkdir(parents=True, exist_ok=True)
DIR_FLAT.mkdir(parents=True, exist_ok=True)

In [2]:
# get final transformation label to dense freesurfer mesh
file_out = DIR_DENSE / f"{HEMI}.composed.label"
file_in = Path(DIR_BASE) / SUBJ / "anatomy/dense_refined" / f"{HEMI}.white_match_final"
file_in2 = Path(DIR_BASE) / SUBJ / "anatomy/gbb" / f"{HEMI}.white_def2_refined"
file_ind2 = Path(DIR_BASE) / SUBJ / "anatomy/gbb" / f"{HEMI}.white_def2_refined_ind.txt"
file_ind3 = Path(DIR_BASE) / SUBJ / "anatomy/dense_epi" / f"{HEMI}.white_def2_ind"
get_composed_label(file_out, file_in, file_in2, file_ind2, file_ind3)

100%|██████████| 294596/294596 [10:39<00:00, 460.93it/s]


In [3]:
# transform arrays
file_ind = DIR_DENSE / f"{HEMI}.composed.label"
file_geom = Path(DIR_BASE) / SUBJ / "anatomy/dense" / f"{HEMI}.white"
file_mgh = []
for sess in SESSION[SUBJ]:
    for day in SESSION[SUBJ][sess]:
        if "_uncorrected" not in sess:
            file_mgh.append(
                Path(DIR_BASE) 
                / SUBJ 
                / f"odc/results/Z/sampled/Z_all_left_right_{sess}{day}" 
                / f"{HEMI}.Z_all_left_right_{sess}{day}_layer_{LAYER}.mgh"
            )

ind = read_label(file_ind)
vtx, _ = read_geometry(file_geom)
for f in file_mgh:
    file_out = DIR_DENSE / f"{Path(f).stem}_dense.mgh"
    arr, _, _ = read_mgh(f)
    res = np.zeros(len(vtx))
    res[ind] = arr
    write_mgh(file_out, res)

/data/pt_01880/source/miniconda3/envs/odc/lib/python3.10/site-packages/nibabel/spatialimages.py:495: UserWarning: Using large vector Freesurfer hack; header will not be compatible with SPM or FSL
  hdr.set_data_shape(shape)


In [4]:
# transform label
file_ind = DIR_DENSE / f"{HEMI}.composed.label"
file_geom = Path(DIR_BASE) / SUBJ / "/data/pt_01880/Experiment1_ODC/p1/anatomy/dense/lh.white"
file_label = []
for label in ["fov", "v1", "v2", "v2a", "v2b", "v3", "v3a", "v3b"]:
    file_label.append(Path(DIR_BASE) / SUBJ / "anatomy/label" / f"{HEMI}.{label}.label")

ind = read_label(file_ind)
for f in file_label:
    file_out = DIR_DENSE / f"{Path(f).stem}_dense.label"
    l_ = read_label(f)
    res = ind[l_]
    write_label(file_out, res)

In [5]:
# get average maps
for sess in SESSION[SUBJ]:
    days = SESSION[SUBJ][sess]
    if "_uncorrected" not in sess:
        file_mgh1 = (
            DIR_DENSE 
            / f"{HEMI}.Z_all_left_right_{sess}{days[0]}_layer_{LAYER}_dense.mgh"
        )
        file_mgh2 = (
            DIR_DENSE
            / f"{HEMI}.Z_all_left_right_{sess}{days[1]}_layer_{LAYER}_dense.mgh"
        )
        file_out = DIR_DENSE / f"{HEMI}.Z_all_left_right_{sess}_layer_{LAYER}_avg.mgh"
        arr1, _, _ = read_mgh(file_mgh1)
        arr2, _, _ = read_mgh(file_mgh2)
        res = (arr1 + arr2) / 2
        write_mgh(file_out, res) 

/data/pt_01880/source/miniconda3/envs/odc/lib/python3.10/site-packages/nibabel/spatialimages.py:495: UserWarning: Using large vector Freesurfer hack; header will not be compatible with SPM or FSL
  hdr.set_data_shape(shape)


In [6]:
# get banpass filtered contrast
file_geom = Path(DIR_BASE) / SUBJ / "anatomy/dense" / f"{HEMI}.white"
file_label = DIR_DENSE / f"{HEMI}.v1_dense.label"
file_mgh = DIR_DENSE / f"{HEMI}.Z_all_left_right_GE_EPI_layer_5_avg.mgh"
vtx, fac = read_geometry(file_geom)
label = read_label(file_label)
arr, _, _ = read_mgh(file_mgh)
mesh = Mesh(vtx, fac)
surf_roi = mesh.remove_vertices(label)
verts = surf_roi[0]
faces = surf_roi[1]
for filter_size in [0.025, 0.1, 2.0]:
    filt = LaplacianGaussian(verts, faces, filter_size)
    tmp = filt.apply(arr[label])
    res = np.zeros(len(vtx))
    res[label] = tmp
    file_out = DIR_DENSE / f"{Path(file_mgh).stem}_bandpass_{filter_size}.mgh"
    write_mgh(file_out, res)

sort faces: 1.0 %
sort faces: 2.0 %
sort faces: 3.0 %
sort faces: 4.0 %
sort faces: 5.0 %
sort faces: 6.0 %
sort faces: 7.0 %
sort faces: 8.0 %
sort faces: 9.0 %
sort faces: 10.0 %
sort faces: 11.0 %
sort faces: 12.0 %
sort faces: 13.0 %
sort faces: 14.0 %
sort faces: 15.0 %
sort faces: 16.0 %
sort faces: 17.0 %
sort faces: 18.0 %
sort faces: 19.0 %
sort faces: 20.0 %
sort faces: 21.0 %
sort faces: 22.0 %
sort faces: 23.0 %
sort faces: 24.0 %
sort faces: 25.0 %
sort faces: 26.0 %
sort faces: 27.0 %
sort faces: 28.0 %
sort faces: 29.0 %
sort faces: 30.0 %
sort faces: 31.0 %
sort faces: 32.0 %
sort faces: 33.0 %
sort faces: 34.0 %
sort faces: 35.0 %
sort faces: 36.0 %
sort faces: 37.0 %
sort faces: 38.0 %
sort faces: 39.0 %
sort faces: 40.0 %
sort faces: 41.0 %
sort faces: 42.0 %
sort faces: 43.0 %
sort faces: 44.0 %
sort faces: 45.0 %
sort faces: 46.0 %
sort faces: 47.0 %
sort faces: 48.0 %
sort faces: 49.0 %
sort faces: 50.0 %
sort faces: 51.0 %
sort faces: 52.0 %
sort faces: 53.0 %
so

In [12]:
# project to patch
file_out = DIR_FLAT / f"{HEMI}.flat"
file_patch = "/data/pt_01880/Experiment1_ODC/p1/anatomy/flat/lh.flat.patch.flat"
patch_as_mesh(file_out, file_patch)

file_mgh = DIR_DENSE.glob("*.mgh")
for f_in in file_mgh:
    f_out = DIR_FLAT / f_in.name
    mgh_to_patch(f_out, f_in, file_patch)

file_out = DIR_FLAT / f"{HEMI}.curv"
file_in = Path(DIR_BASE) / SUBJ / "anatomy/dense" / f"{HEMI}.curv"
curv_to_patch(file_out, file_in, file_patch)

label = ["v1", "v2", "v2a", "v2b", "v3", "v3a", "v3b"]
for l_ in label:
    f_out = DIR_FLAT / f"{HEMI}.{l_}_flat.label"
    f_in = DIR_DENSE / f"{HEMI}.{l_}_dense.label"
    label_to_patch(f_out, f_in, file_patch)

/data/pt_01880/source/miniconda3/envs/odc/lib/python3.10/site-packages/nibabel/analyze.py:401: UserWarning: Using large vector Freesurfer hack; header will not be compatible with SPM or FSL
  obj.set_data_shape(header.get_data_shape())
/data/pt_01880/source/miniconda3/envs/odc/lib/python3.10/site-packages/nibabel/spatialimages.py:495: UserWarning: Using large vector Freesurfer hack; header will not be compatible with SPM or FSL
  hdr.set_data_shape(shape)
